### - 1 in every 8 people in the world live with a mental disorder
### - The majority of suicides worldwide are related to mental health disorders; depression, substance use, and psychosis
### - The World Health Organization (WHO) reports that suicide is one of the leading causes of death among individuals aged 15-19

## The Idea:

## Show ingishts about what people suffering from those disorders are talking about through topic modelling

## Build a mental health recommendation system offering personalized advice derived from shared community experiences of like-minded people by leveraging text analytics, NLP and LLM.

## Recommender System

In [1]:
from google.colab import drive
import sys
drive.mount('/content/drive')
%cd "/content/drive/My Drive/Colab Notebooks/mental-health-codes"
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/mental-health-codes')
#import utilities as ut

%reload_ext autoreload
%autoreload 2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/mental-health-codes


In [2]:
#Importing necessary libraries
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer, word_tokenize
from nltk.stem import WordNetLemmatizer
import torch
from transformers import BertTokenizer, BertModel
from transformers import pipeline
import io

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


#### We are scraping data from a medical forum for 5 mental disorders people are talking about. ####

#### We have the tag(disorder), title of the concern, the user concerns, and the suggestions offered by people for that concern ####

In [3]:
import requests
import re
import urllib.request
from urllib.parse import urlparse
import unicodedata
from urllib.parse import urljoin
import logging
from lxml.html import fromstring

# Insights about disorders based on Topic Modelling

In [4]:
!pip install bertopic

In [4]:
from bertopic import BERTopic

df=pd.read_csv('explorations-reddit.csv')
df.head(5)

,sp_id,rp_id,seeker_post,response_post,level,rationales
0,65m92s,dgbdk7z,Help. Help me. I dunno what I'm doing anymore,"That's pretty vague, do you not know what you'...",2,do you not know what you're doing in regards t...
1,9ezsfi,e5t3oxh,I'm done saying I love you to her because I do...,idk what a Red pill means exactly but my advic...,0,NaN
2,6b2cmc,dhj8tcb,Always feel like I'm being criticized and mock...,"I think it's social anxiety , that creates par...",0,NaN
3,8iz0as,dyvq1ne,My diet becomes fucked when i get depressed.. ...,By any chance do you think you're in a loop. J...,2,By any chance do you think you're in a loop|
4,aow3l9,eg40ecq,I hate not knowing why. I was diagnosed with d...,depression. not sadness which is caused by som...,0,NaN


In [6]:
df.drop_duplicates(subset='seeker_post', keep='first', inplace=True)
df[:5]

,sp_id,rp_id,seeker_post,response_post,level,rationales
0,65m92s,dgbdk7z,Help. Help me. I dunno what I'm doing anymore,"That's pretty vague, do you not know what you'...",2,do you not know what you're doing in regards t...
1,9ezsfi,e5t3oxh,I'm done saying I love you to her because I do...,idk what a Red pill means exactly but my advic...,0,NaN
2,6b2cmc,dhj8tcb,Always feel like I'm being criticized and mock...,"I think it's social anxiety , that creates par...",0,NaN
3,8iz0as,dyvq1ne,My diet becomes fucked when i get depressed.. ...,By any chance do you think you're in a loop. J...,2,By any chance do you think you're in a loop|
4,aow3l9,eg40ecq,I hate not knowing why. I was diagnosed with d...,depression. not sadness which is caused by som...,0,NaN


In [5]:
import nltk
nltk.download('punkt_tab')
def remove_stopwords(text):
    words = nltk.word_tokenize(text)
    stop_words = set(stopwords.words("english"))
    filtered_words = [word for word in words if word.lower() not in stop_words]
    filtered_words = [word for word in filtered_words if word.isalnum()]
    return " ".join(filtered_words)

df['seeker_post'] = df['seeker_post'].astype(str).apply(remove_stopwords)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [6]:
df_lvl_0= df[df['level']==0].reset_index(drop=True)
df_lvl_1= df[df['level']==1].reset_index(drop=True)
df_lvl_2= df[df['level']==2].reset_index(drop=True)

In [8]:
docs = df_lvl_0['seeker_post'].to_list()
model = BERTopic(verbose=True)
topics, probabilities = model.fit_transform(docs)
model.get_topic_freq().head(11)


2024-11-23 16:34:30,593 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/82 [00:00<?, ?it/s]

2024-11-23 16:35:06,976 - BERTopic - Embedding - Completed ✓
2024-11-23 16:35:06,978 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-11-23 16:35:36,679 - BERTopic - Dimensionality - Completed ✓
2024-11-23 16:35:36,681 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-11-23 16:35:36,808 - BERTopic - Cluster - Completed ✓
2024-11-23 16:35:36,816 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-11-23 16:35:36,935 - BERTopic - Representation - Completed ✓


,Topic,Count
1,-1,1126
2,0,268
3,1,116
12,2,101
11,3,83
28,4,53
5,5,50
0,6,47
22,7,45
17,8,40


In [9]:
model.get_topic_info()


,Topic,Count,Name,Representation,Representative_Docs
0,-1,1126,-1_feel_want_like_people,"[feel, want, like, people, life, anyone, know,...",[anyone else feel like cant anything feel like...
1,0,268,0_depression_depressed_like_know,"[depression, depressed, like, know, feel, thin...",[open depression social anxiety family thinkin...
2,1,116,1_school_college_motivation_work,"[school, college, motivation, work, study, hom...","[one anyone going school work, barely class 2 ..."
3,2,101,2_sleep_bed_tired_wake,"[sleep, bed, tired, wake, awake, ca, day, nigh...",[ever terrible day feel tired night ca even at...
4,3,83,3_suicide_suicidal_way_thoughts,"[suicide, suicidal, way, thoughts, attempt, wa...",[know Ever get feeling way die suicide know ac...
5,4,53,4_meds_zoloft_lexapro_wellbutrin,"[meds, zoloft, lexapro, wellbutrin, taking, pr...",[Question Lexapro SSRI Hey guys started Lexapr...
6,5,50,5_kill_die_please_life,"[kill, die, please, life, wan, live, want, na,...","[Kill nothing else live, wan na kill kill holy..."
7,6,47,6_love_versions_ex_girl,"[love, versions, ex, girl, miss, months, gon, ...",[ca love someone love think great advice peopl...
8,7,45,7_cry_numb_crying_feel,"[cry, numb, crying, feel, emotions, cried, ca,...",[want cry ca get people easily cry time seems ...
9,8,40,8_birthday_today_happy_year,"[birthday, today, happy, year, turning, bday, ...",[19th Birthday come every year birthday cry mu...


In [10]:
model.visualize_barchart()

In [11]:
docs = df_lvl_1['seeker_post'].to_list()
model = BERTopic(verbose=True)
topics, probabilities = model.fit_transform(docs)
model.get_topic_freq().head(11)

2024-11-23 16:35:56,596 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

2024-11-23 16:35:58,816 - BERTopic - Embedding - Completed ✓
2024-11-23 16:35:58,818 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-11-23 16:35:59,011 - BERTopic - Dimensionality - Completed ✓
2024-11-23 16:35:59,014 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-11-23 16:35:59,024 - BERTopic - Cluster - Completed ✓
2024-11-23 16:35:59,031 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-11-23 16:35:59,051 - BERTopic - Representation - Completed ✓


,Topic,Count
1,-1,48
3,0,22
0,1,18
2,2,16


In [12]:
model.get_topic_info()


,Topic,Count,Name,Representation,Representative_Docs
0,-1,48,-1_help_feel_know_like,"[help, feel, know, like, life, want, please, f...",[Seeing pretty girl makes guy sad attractive s...
1,0,22,0_depressed_feel_depression_stuck,"[depressed, feel, depression, stuck, past, lik...",[meds years Always feel like never work quit H...
2,1,18,1_day_today_worst_ever,"[day, today, worst, ever, get, time, life, one...",[worst fucking day ever Arghhhhhhh god worst f...
3,2,16,2_want_kill_ca_anymore,"[want, kill, ca, anymore, even, suicide, die, ...","[anymore much going head handle want stop, wan..."


In [13]:
model.visualize_barchart()

In [14]:
docs = df_lvl_2['seeker_post'].to_list()
model = BERTopic(verbose=True)
topics, probabilities = model.fit_transform(docs)
model.get_topic_freq().head(11)


2024-11-23 16:36:04,553 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/12 [00:00<?, ?it/s]

2024-11-23 16:36:12,744 - BERTopic - Embedding - Completed ✓
2024-11-23 16:36:12,747 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-11-23 16:36:13,432 - BERTopic - Dimensionality - Completed ✓
2024-11-23 16:36:13,434 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-11-23 16:36:13,458 - BERTopic - Cluster - Completed ✓
2024-11-23 16:36:13,466 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-11-23 16:36:13,520 - BERTopic - Representation - Completed ✓


,Topic,Count
1,-1,134
0,0,110
2,1,72
4,2,29
3,3,21
5,4,10


In [15]:
model.get_topic_info()


,Topic,Count,Name,Representation,Representative_Docs
0,-1,134,-1_feel_want_feeling_life,"[feel, want, feeling, life, like, nothing, eve...",[go dream school depressed ass even care much ...
1,0,110,0_life_want_die_suicide,"[life, want, die, suicide, feel, kill, like, p...",[Sadness often caused discrepancy want Help li...
2,1,72,1_depression_feel_like_depressed,"[depression, feel, like, depressed, know, real...",[feel like depressed hard even pretend happy a...
3,2,29,2_get_bed_tired_ca,"[get, bed, tired, ca, classes, fucking, sleep,...",[ca keep school ca keep classes everyone aroun...
4,3,21,3_friends_alone_trust_people,"[friends, alone, trust, people, know, humans, ...",[Literally friends know interact people single...
5,4,10,4_tears_cried_crying_day,"[tears, cried, crying, day, reason, shower, bu...",[video time tears Watch Schools Kill Ken Robin...


In [16]:
model.visualize_barchart()

# Recommender System

In [17]:
#We can take in a user input in this form

# disorder_types = input("0", "1", "2")

disorder_level= 2

In [18]:
#This too should be an user input

user_rant="Hi everyone,,This year has been hell for me. I developed social anxiety and I forgot how miserable this makes me feel. I feel like I'm losing my sanity and I don't cope well. Anything can trigger it and it depresses me after the panic attack subsides. All I want to do is sleep. My husband found this website for me because I don't do social media. My therapist said to me that I need out patient intensive therapy treatment or in patient therapy. Even that is depressing to hear from her. I don't like my meds because of the altered state if mind it creates."

In [19]:
df=pd.read_csv('explorations-reddit.csv')
df.head()

,sp_id,rp_id,seeker_post,response_post,level,rationales
0,65m92s,dgbdk7z,Help. Help me. I dunno what I'm doing anymore,"That's pretty vague, do you not know what you'...",2,do you not know what you're doing in regards t...
1,9ezsfi,e5t3oxh,I'm done saying I love you to her because I do...,idk what a Red pill means exactly but my advic...,0,NaN
2,6b2cmc,dhj8tcb,Always feel like I'm being criticized and mock...,"I think it's social anxiety , that creates par...",0,NaN
3,8iz0as,dyvq1ne,My diet becomes fucked when i get depressed.. ...,By any chance do you think you're in a loop. J...,2,By any chance do you think you're in a loop|
4,aow3l9,eg40ecq,I hate not knowing why. I was diagnosed with d...,depression. not sadness which is caused by som...,0,NaN


In [20]:
def remove_stopwords(text):
    words = nltk.word_tokenize(text)
    stop_words = set(stopwords.words("english"))
    filtered_words = [word for word in words if word.lower() not in stop_words]
    filtered_words = [word for word in filtered_words if word.isalnum()]
    return " ".join(filtered_words)

# Assuming df['Concern'] contains your text data
df['seeker_post'] = df['seeker_post'].str.lower()
print(df['seeker_post'])

0           help. help me. i dunno what i'm doing anymore
1       i'm done saying i love you to her because i do...
2       always feel like i'm being criticized and mock...
3       my diet becomes fucked when i get depressed.. ...
4       i hate not knowing why. i was diagnosed with d...
                              ...                        
3079    does anyone else keep forgetting stuff the nee...
3080    what does depression feel like?. honest questi...
3081    i'm to scared to commit suicide.. all i can fe...
3082    i just want to disappear but i don't want to h...
3083    26 year old male, living at home, low income j...
Name: seeker_post, Length: 3084, dtype: object


In [21]:
df_disorder=df[df["level"]==2]

In [22]:
df_disorder= df_disorder[['seeker_post']].drop_duplicates().reset_index(drop=True)
df_disorder['User Concern']=user_rant

In [23]:
df_disorder

,seeker_post,User Concern
0,help. help me. i dunno what i'm doing anymore,"Hi everyone,,This year has been hell for me. I..."
1,my diet becomes fucked when i get depressed.. ...,"Hi everyone,,This year has been hell for me. I..."
2,"lost my job, have no reason to live. my career...","Hi everyone,,This year has been hell for me. I..."
3,why the fuck am i depressed. i don't even reme...,"Hi everyone,,This year has been hell for me. I..."
4,why does death make people sad?. a relief from...,"Hi everyone,,This year has been hell for me. I..."
...,...,...
368,feeling numb. title really says it all. just f...,"Hi everyone,,This year has been hell for me. I..."
369,it's so depressing to know that i have an adhd...,"Hi everyone,,This year has been hell for me. I..."
370,why am i depressed?. why am i depressed? &amp...,"Hi everyone,,This year has been hell for me. I..."
371,i feel like i’m an inconvenience or a nuisance...,"Hi everyone,,This year has been hell for me. I..."


In [24]:
!pip install vaderSentiment

In [25]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Calculate the sentiment score for the single "User Concern" value
user_concern_sentiment = analyzer.polarity_scores(df_disorder.iloc[0]['User Concern'])['compound']

# Apply the sentiment score to all rows in the "Concern" column

df_disorder['Concern_Sentiment'] = df_disorder['seeker_post'].apply(lambda x: analyzer.polarity_scores(x)['compound'])
df_disorder['User_Concern_Sentiment'] = user_concern_sentiment

# Print the updated DataFrame
df_disorder

,seeker_post,User Concern,Concern_Sentiment,User_Concern_Sentiment
0,help. help me. i dunno what i'm doing anymore,"Hi everyone,,This year has been hell for me. I...",0.6597,-0.9695
1,my diet becomes fucked when i get depressed.. ...,"Hi everyone,,This year has been hell for me. I...",-0.8750,-0.9695
2,"lost my job, have no reason to live. my career...","Hi everyone,,This year has been hell for me. I...",-0.8308,-0.9695
3,why the fuck am i depressed. i don't even reme...,"Hi everyone,,This year has been hell for me. I...",-0.9403,-0.9695
4,why does death make people sad?. a relief from...,"Hi everyone,,This year has been hell for me. I...",-0.8591,-0.9695
...,...,...,...,...
368,feeling numb. title really says it all. just f...,"Hi everyone,,This year has been hell for me. I...",-0.3022,-0.9695
369,it's so depressing to know that i have an adhd...,"Hi everyone,,This year has been hell for me. I...",-0.5407,-0.9695
370,why am i depressed?. why am i depressed? &amp...,"Hi everyone,,This year has been hell for me. I...",-0.7882,-0.9695
371,i feel like i’m an inconvenience or a nuisance...,"Hi everyone,,This year has been hell for me. I...",-0.8339,-0.9695


In [26]:
# We assume that a sentiment score of greater than 0.05 (based on Vader) of the user rant means the user is talking about something cheerful

# In other words, we need to keep the user rant if its negative or neutral
# Same for the concerns in our data

df_disorder= df_disorder[(df_disorder['Concern_Sentiment']<0.05) & (df_disorder['User_Concern_Sentiment']<0.05)]

In [30]:
#Applying word vector based similarity and finding score of all Scraped concerns with the user input
import spacy.cli
spacy.cli.download("en_core_web_md")

nlp = spacy.load('en_core_web_md')
def word_vector_similarity(text1, text2):
    doc1 = nlp(text1)
    doc2 = nlp(text2)
    return doc1.similarity(doc2)


# this adds the simlarity score of each comment in the concern column with the user input
df_disorder['Similarity Score'] = df_disorder['seeker_post'].apply(word_vector_similarity, text2=user_rant)

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [31]:
#Similarity Score Threshold setting
df_disorder=df_disorder[df_disorder['Similarity Score']>0.5].sort_values(by='Similarity Score', ascending=False).reset_index(drop=True)[:10]

In [34]:
df_sug= df_disorder.merge(df, on='seeker_post', how='left')[['seeker_post','User Concern','Similarity Score','Concern_Sentiment','User_Concern_Sentiment','response_post']]
df_sug['response_post']= df_sug['response_post'].astype(str)

# Combine rows into one paragraph
paragraph = ' '.join(df_sug['response_post'])

# Load a pre-trained spaCy model (English)
nlp = spacy.load('en_core_web_md')
nlp.max_length = 2000000 # or even higher

# Split the paragraph into sentences
doc = nlp(paragraph)
sentences = [sent.text for sent in doc.sents]

In [35]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

df_sug = df_disorder.merge(df, on='seeker_post', how='left')[['seeker_post', 'User Concern', 'Similarity Score', 'Concern_Sentiment', 'User_Concern_Sentiment', 'response_post']]
df_sug['response_post'] = df_sug['response_post'].astype(str)

# Combine rows into one paragraph
paragraph = ' '.join(df_sug['response_post'])

# Load a pre-trained spaCy model (English)
nlp = spacy.load('en_core_web_md')
nlp.max_length = 2000000  # or even higher

# Split the paragraph into sentences
doc = nlp(paragraph)
sentences = [sent.text for sent in doc.sents]

# Initialize a matrix to store sentence similarity scores
num_sentences = len(sentences)
similarity_matrix = np.zeros((num_sentences, num_sentences))

# Calculate sentence similarity using cosine similarity
for i in range(num_sentences):
    for j in range(i, num_sentences):  # Only calculate each pair once
        if i != j:
            vector1 = nlp(sentences[i]).vector
            vector2 = nlp(sentences[j]).vector
            similarity = cosine_similarity([vector1], [vector2])[0][0]
            similarity_matrix[i, j] = similarity
            similarity_matrix[j, i] = similarity  # Symmetric

# Print the sentence similarity matrix (optional)
print("Sentence Similarity Matrix:")
print(similarity_matrix)

Sentence Similarity Matrix:
[[0.         0.77943271 0.85810679 ... 0.85564995 0.88207054 0.26684383]
 [0.77943271 0.         0.6343984  ... 0.64951819 0.80066895 0.18108521]
 [0.85810679 0.6343984  0.         ... 0.87712693 0.78653735 0.30988312]
 ...
 [0.85564995 0.64951819 0.87712693 ... 0.         0.87929726 0.31296578]
 [0.88207054 0.80066895 0.78653735 ... 0.87929726 0.         0.24523108]
 [0.26684383 0.18108521 0.30988312 ... 0.31296578 0.24523108 0.        ]]


In [57]:
# Sum up the similarity values for each sentence
sentence_similarity_scores = np.sum(similarity_matrix, axis=1)

# Create a list of (sentence, score) tuples
sentence_scores_tuples = [(sentence, score) for sentence, score in zip(sentences, sentence_similarity_scores)]

# Sort the tuples by score in descending order
sentence_scores_tuples_sorted = sorted(sentence_scores_tuples, key=lambda x: x[1], reverse=True)

# Specify the desired percentage, e.g., 10%
percentage = 10

# Calculate the threshold score based on the desired percentage (e.g., 10%)
percentage = 10
num_sentences = len(sentences)
threshold_score = np.percentile(sentence_similarity_scores, 100 - percentage)

# Create a list of sentences that have scores greater than or equal to the threshold
top_10_sentences = [sentence for sentence, score in sentence_scores_tuples if score >= threshold_score]

# Print the top 10% of sentences
for i, sentence in enumerate(top_10_sentences, 1):
    print(f"Top {i}: {sentence}")

Top 1: They might help you better understand your emotions so that you can control them in the future.
Top 2: Maybe you think it's because they just want to be nice or maybe it's because of other reasons,
Top 3: I'm not saying it's easy, or that you even can.
Top 4: I took hold of this and I try to spend time with them the best I can, and sometimes I won't because I'm in a state of self pity, but I'll realize what I'm doing and pull myself out of my bed and spend some time with them.
Top 5: I understand how you feel, but there has to be at least one thing that makes you feel in a non hurtful way.
Top 6: It may not come right away, but I'm sure you'll think of something if you really try hard.


In [58]:
# Function to filter sentences with non-negative sentiment
def filter_positive_sentences(sentences):
    positive_sentences = []
    for sentence in sentences:
        sentiment = analyzer.polarity_scores(sentence)
        if sentiment['compound'] >= 0:
            positive_sentences.append(sentence)
    return positive_sentences

# Filter the list of sentences
top_10_sentences = filter_positive_sentences(top_10_sentences)

# Print the updated list of positive sentences
for i, sentence in enumerate(top_10_sentences, 1):
    print(f"Positive {i}: {sentence}")

Positive 1: They might help you better understand your emotions so that you can control them in the future.
Positive 2: Maybe you think it's because they just want to be nice or maybe it's because of other reasons,
Positive 3: I took hold of this and I try to spend time with them the best I can, and sometimes I won't because I'm in a state of self pity, but I'll realize what I'm doing and pull myself out of my bed and spend some time with them.
Positive 4: It may not come right away, but I'm sure you'll think of something if you really try hard.


In [59]:
#Just limiting the number of sentences to summarize for summarization speed without GPU
if len(top_10_sentences)>25:
    top_10_sentences=top_10_sentences[:25]
top_10_sentences

['They might help you better understand your emotions so that you can control them in the future.',
 "Maybe you think it's because they just want to be nice or maybe it's because of other reasons,",
 "I took hold of this and I try to spend time with them the best I can, and sometimes I won't because I'm in a state of self pity, but I'll realize what I'm doing and pull myself out of my bed and spend some time with them.",
 "It may not come right away, but I'm sure you'll think of something if you really try hard."]

In [63]:
# Using BART tuned for text-summarization through transformers pipelines

from transformers import pipeline

# Create the Summarization Pipeline
summarizer = pipeline("summarization")

# Input text to be summarized
text = ' '.join(top_10_sentences)

# Generate a summary
summary = summarizer(text, max_length=76, min_length=30, do_sample=False)


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [64]:
print(summary)

[{'summary_text': " Sometimes I won't because I'm in a state of self pity, but I'll pull myself out of my bed and spend some time with them . They might help you better understand your emotions so that you can control them in the future ."}]


In [65]:
# Print the summary
print("Here is a summarized form of what people are sugeesting to people with similar concerns:\n\n",summary[0]['summary_text'])

Here is a summarized form of what people are sugeesting to people with similar concerns:

  Sometimes I won't because I'm in a state of self pity, but I'll pull myself out of my bed and spend some time with them . They might help you better understand your emotions so that you can control them in the future .
